<a href="https://colab.research.google.com/github/Multik838/-Data-analysis-algorithms/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%A3%D1%80%D0%BE%D0%BA_5_HW_wf_udf_joins%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 51.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ad3152e040cc3ae029474f82057b986220310be07cf4264ea04480eb3cfc40f7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

# Самостоятельная работа к уроку 3
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

In [3]:
from google.colab import drive
drive.mount('/content/drive')

raw_sales = spark.read.format('csv').option('header', 'True').load('/content/raw_sales.csv') 

Mounted at /content/drive


In [4]:
#df = spark.read.csv('raw_sales.csv', inferShema=True, header=True)

In [5]:
raw_sales.show(1)

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-02-07 00:00:00|    2607|525000|       house|       4|
+-------------------+--------+------+------------+--------+
only showing top 1 row



In [15]:
#Импортируем необходимые библиотеки
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.sql.functions import max as max_

In [ ]:
#F.lit()

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего (1 балл)


In [16]:
windSpec_10before = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold') \
    .rowsBetween(Window.currentRow - 11, Window.currentRow - 1)

In [17]:
raw_sales.withColumn('Средняя стомость 10 проданных домов до текущего в том же районе', F.avg('price').over(windSpec_10before)).show()

+-------------------+--------+-------+------------+--------+---------------------------------------------------------------+
|           datesold|postcode|  price|propertyType|bedrooms|Средняя стомость 10 проданных домов до текущего в том же районе|
+-------------------+--------+-------+------------+--------+---------------------------------------------------------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                                                           null|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|                                                       327000.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|                                                       558500.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|                                              647333.3333333334|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|                                                       564250.0|


In [18]:
windSpec_10after = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold') \
    .rowsBetween(Window.currentRow - 10, Window.currentRow - 0)

In [19]:
raw_sales.withColumn('Средняя стомость 10 проданных домов после текущего в том же районе', F.avg('price').over(windSpec_10after)).show()

+-------------------+--------+-------+------------+--------+------------------------------------------------------------------+
|           datesold|postcode|  price|propertyType|bedrooms|Средняя стомость 10 проданных домов после текущего в том же районе|
+-------------------+--------+-------+------------+--------+------------------------------------------------------------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                                                          327000.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|                                                          558500.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|                                                 647333.3333333334|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|                                                          564250.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|                                            

In [20]:
#Ищем дату по типу собственности дом

In [21]:
filter = (raw_sales.select('propertyType', 'datesold')\
.filter('propertyType = "house"')\
.groupBy('propertyType')
.agg(max_('datesold')))

filter.show()

+------------+-------------------+
|propertyType|      max(datesold)|
+------------+-------------------+
|       house|2019-07-27 00:00:00|
+------------+-------------------+



In [22]:
#Выводим район, цена, дата, тип собственности

In [23]:
raw_sales.select('price', 'propertyType','datesold','postcode')\
.filter('propertyType = "house"')\
.filter('datesold = "2019-07-27 00:00:00"')\
.orderBy('postcode','price')\
.show()

+------+------------+-------------------+--------+
| price|propertyType|           datesold|postcode|
+------+------------+-------------------+--------+
|647500|       house|2019-07-27 00:00:00|    2615|
|610000|       house|2019-07-27 00:00:00|    2905|
|865000|       house|2019-07-27 00:00:00|    2905|
+------+------------+-------------------+--------+



## Задание 2
Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)


*(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)*

In [26]:
df = spark.read.csv('raw_sales.csv', inferSchema=True, header=True)

In [27]:
windSpec_10before = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold') \
    .rowsBetween(Window.currentRow - 11, Window.currentRow - 1)

df.withColumn('Средняя стомость 10 проданных домов до текущего в том же районе', F.avg('price').over(windSpec_10before)).show()

+-------------------+--------+-------+------------+--------+---------------------------------------------------------------+
|           datesold|postcode|  price|propertyType|bedrooms|Средняя стомость 10 проданных домов до текущего в том же районе|
+-------------------+--------+-------+------------+--------+---------------------------------------------------------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                                                           null|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|                                                       327000.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|                                                       558500.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|                                              647333.3333333334|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|                                                       564250.0|


In [30]:
#И выводим среднюю и id


df.groupBy(F.year('datesold')).avg().show()

+--------------+------------------+-----------------+------------------+
|year(datesold)|     avg(postcode)|       avg(price)|     avg(bedrooms)|
+--------------+------------------+-----------------+------------------+
|          2007| 2737.292517006803|522377.2108843537| 3.292517006802721|
|          2018|2725.8535510627266|660701.0427682737|3.0907205806117157|
|          2015|  2722.73711622807|626101.3426535088|3.3209978070175437|
|          2013| 2734.596979707409|553416.2968381312|3.2987258140632374|
|          2014| 2727.868319944115|  592653.76143905|3.3709395738735592|
|          2019| 2735.301805054152|634184.2187725632|  3.11913357400722|
|          2012|2732.2093649085036|552501.3692142088| 3.279870828848224|
|          2009| 2745.084852734923|496092.0329593268| 3.279102384291725|
|          2016|2725.5798362333676|635185.3116683726|3.2315762538382806|
|          2010|2744.0456591639872|559564.8102893891|  3.32540192926045|
|          2011| 2738.410900183711|566715.110226576

## Задание 3
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)

+--------------+------------------+-----------------+------------------+
|year(datesold)|     avg(postcode)|       avg(price)|     avg(bedrooms)|
+--------------+------------------+-----------------+------------------+
|          2007| 2737.292517006803|522377.2108843537| 3.292517006802721|
|          2018|2725.8535510627266|660701.0427682737|3.0907205806117157|
|          2015|  2722.73711622807|626101.3426535088|3.3209978070175437|
|          2013| 2734.596979707409|553416.2968381312|3.2987258140632374|
|          2014| 2727.868319944115|  592653.76143905|3.3709395738735592|
|          2019| 2735.301805054152|634184.2187725632|  3.11913357400722|
|          2012|2732.2093649085036|552501.3692142088| 3.279870828848224|
|          2009| 2745.084852734923|496092.0329593268| 3.279102384291725|
|          2016|2725.5798362333676|635185.3116683726|3.2315762538382806|
|          2010|2744.0456591639872|559564.8102893891|  3.32540192926045|
|          2011| 2738.410900183711|566715.110226576